<a href="https://colab.research.google.com/github/amanmehra-23/RE-Id_RP/blob/main/Re_IDSimple1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch_geometric

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torchvision import models, transforms
from torch_geometric.nn import GCNConv
from torch.utils.data import Dataset, DataLoader
from PIL import Image

In [ ]:
# --------------------------
# 1. Define the Dataset Class
# --------------------------
class Market1501Dataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (str): Directory with images (e.g., bounding_box_train, bounding_box_test, or query).
            transform: Preprocessing transforms.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        for file in os.listdir(root_dir):
            if file.endswith('.jpg'):
                # Expected filename: "0002_c1s1_000451_03.jpg"
                id_str = file.split('_')[0]
                if id_str.startswith('-') or not id_str.isdigit():
                    continue  # Skip junk/distractor images
                person_id = int(id_str)
                if person_id <= 0:
                    continue
                self.image_paths.append(os.path.join(root_dir, file))
                self.labels.append(person_id)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


In [ ]:
# EfficientNet-B4 Backbone (removes the classification head)
class EfficientNetBackbone(nn.Module):
    def __init__(self, pretrained=True):
        super(EfficientNetBackbone, self).__init__()
        effnet = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1 if pretrained else None)
        self.features = nn.Sequential(
            effnet.features  # This includes all convolutional layers before avgpool and fc
        )

    def forward(self, x):
        # Expected output: (B, 1792, 7, 7) for 224x224 input
        return self.features(x)


In [ ]:
# Build Grid Graph for the feature map from the CNN
def build_grid_edge_index(grid_size):
    """
    Constructs edge indices for a grid graph.
    Each node (patch) is connected to its right and down neighbors (and vice versa).
    """
    H, W = grid_size
    edges = []
    for i in range(H):
        for j in range(W):
            idx = i * W + j
            # Connect to right neighbor, if available
            if j + 1 < W:
                right_idx = i * W + (j + 1)
                edges.append((idx, right_idx))
                edges.append((right_idx, idx))
            # Connect to down neighbor, if available
            if i + 1 < H:
                down_idx = (i + 1) * W + j
                edges.append((idx, down_idx))
                edges.append((down_idx, idx))
    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    return edge_index


In [ ]:
# GNN Branch that processes the CNN feature map
class GNNBranch(nn.Module):
    def __init__(self, in_channels=2048, hidden_channels=512, out_channels=256, grid_size=(7,7)):
        super(GNNBranch, self).__init__()
        self.grid_size = grid_size
        self.edge_index = build_grid_edge_index(grid_size)  # fixed graph structure

        # Two GCN layers
        self.gcn1 = GCNConv(in_channels, hidden_channels)
        self.gcn2 = GCNConv(hidden_channels, out_channels)
        # An additional projection layer for refinement (optional)
        self.fc = nn.Linear(out_channels, out_channels)

    def forward(self, x):
        """
        Args:
            x: Feature map of shape (B, 2048, 7, 7).
        Returns:
            Embedding tensor of shape (B, out_channels) [e.g., (B, 256)].
        """
        B, C, H, W = x.shape
        N = H * W  # number of nodes, e.g., 7*7 = 49
        # Reshape feature map to nodes: (B, N, C)
        x_nodes = x.view(B, C, N).permute(0, 2, 1)
        embeddings = []
        edge_index = self.edge_index.to(x.device)
        for i in range(B):
            node_feat = x_nodes[i]  # shape (N, C)
            h = F.relu(self.gcn1(node_feat, edge_index))
            h = self.gcn2(h, edge_index)
            pooled = h.mean(dim=0)  # global mean pooling (N, ) -> (C_out,)
            embeddings.append(pooled)
        embeddings = torch.stack(embeddings, dim=0)
        embeddings = self.fc(embeddings)
        return embeddings  # shape (B, out_channels)


In [ ]:
class SimpleReIDModel(nn.Module):
    def __init__(self, device='cuda'):
        super(SimpleReIDModel, self).__init__()
        self.device = device
        self.backbone = EfficientNetBackbone(pretrained=True)
        self.gnn_branch = GNNBranch(in_channels=1280, hidden_channels=512, out_channels=256, grid_size=(7,7))

    def forward(self, x):
        x = x.to(self.device)
        feature_map = self.backbone(x)  # shape (B, 1792, 7, 7)
        embedding = self.gnn_branch(feature_map)
        return embedding


In [ ]:
# --------------------------
# 4. Define the Supervised Contrastive Loss
# --------------------------
def supervised_contrastive_loss(embeddings, labels, temperature=0.07):
    """
    Computes the supervised contrastive loss. Embeddings with the same label are pulled together,
    while embeddings with different labels are pushed apart.

    Args:
        embeddings: Tensor of shape (B, D) [B: batch size, D: embedding dimension].
        labels: Tensor of shape (B,) with the identity labels.
        temperature: Scaling factor for the logits.
    Returns:
        A scalar loss value.
    """
    device = embeddings.device
    batch_size = embeddings.shape[0]

    # Normalize embeddings to unit norm
    embeddings = F.normalize(embeddings, p=2, dim=1)

    # Compute cosine similarity matrix, shape: (B, B)
    similarity_matrix = torch.matmul(embeddings, embeddings.T)
    logits = similarity_matrix / temperature

    # Create a binary mask where mask[i, j]=1 if labels[i]==labels[j] and i != j
    labels = labels.contiguous().view(-1, 1)
    mask = torch.eq(labels, labels.T).float().to(device)
    logits_mask = torch.ones_like(mask) - torch.eye(batch_size, device=device)
    mask = mask * logits_mask

    # Calculate log probabilities
    exp_logits = torch.exp(logits) * logits_mask
    denominator = exp_logits.sum(1, keepdim=True) + 1e-8
    log_prob = logits - torch.log(denominator)

    mean_log_prob_pos = (mask * log_prob).sum(1) / (mask.sum(1) + 1e-8)
    loss = -mean_log_prob_pos.mean()
    return loss


In [ ]:
import kagglehub
import shutil
import os

# Download dataset
path = kagglehub.dataset_download("pengcw1/market-1501")
print("Original path:", path)

# Desired destination directory in your Colab workspace
destination_path = "/content/data/market-1501"

# Move the dataset to your preferred path
if not os.path.exists(destination_path):
    shutil.copytree(path, destination_path)

print("✅ Dataset moved to:", destination_path)


Original path: /kaggle/input/market-1501
✅ Dataset moved to: /content/data/market-1501


In [ ]:
# --------------------------
# 5. Setup the Dataset and DataLoader
# --------------------------
# Define a preprocessing pipeline (same for training and evaluation)
preprocess_pipeline = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Set your dataset paths (adjust these based on your environment)
# For example, assume you have a folder structure with "bounding_box_train" for training.
dataset_path = "/kaggle/input/market-1501/Market-1501-v15.09.15"  # Adjust if necessary
train_dir = os.path.join(dataset_path, "bounding_box_train")

# Create the dataset and dataloader
train_dataset = Market1501Dataset(root_dir=train_dir, transform=preprocess_pipeline)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)


In [ ]:
# --------------------------
# 6. Training Loop
# --------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SimpleReIDModel(device=device).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 10  # Set as needed

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        embeddings = model(images)  # (B, 256)
        loss = supervised_contrastive_loss(embeddings, labels, temperature=0.07)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - Supervised Contrastive Loss: {avg_loss:.4f}")

# Optionally, save your model once training is complete:
torch.save(model.state_dict(), "/content/simple_reid_model.pth")


Epoch 1/10 - Supervised Contrastive Loss: 0.1297
Epoch 2/10 - Supervised Contrastive Loss: 0.0904
Epoch 3/10 - Supervised Contrastive Loss: 0.0849
Epoch 4/10 - Supervised Contrastive Loss: 0.0644
Epoch 5/10 - Supervised Contrastive Loss: 0.0469
Epoch 6/10 - Supervised Contrastive Loss: 0.0423
Epoch 7/10 - Supervised Contrastive Loss: 0.0431
Epoch 8/10 - Supervised Contrastive Loss: 0.0419
Epoch 9/10 - Supervised Contrastive Loss: 0.0386
Epoch 10/10 - Supervised Contrastive Loss: 0.0366


In [ ]:
# --------------------------
# 7. (Optional) Evaluation Functions
# --------------------------
def extract_embeddings(model, data_loader, device):
    model.eval()
    all_embeddings = []
    all_labels = []
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            embeddings = model(images)
            all_embeddings.append(embeddings.cpu())
            all_labels.extend(labels.numpy())
    all_embeddings = torch.cat(all_embeddings, dim=0)
    return all_embeddings, np.array(all_labels)

def compute_distance_matrix(query_emb, gallery_emb, metric='euclidean'):
    if metric == 'euclidean':
        return torch.cdist(query_emb, gallery_emb, p=2)
    elif metric == 'cosine':
        q_norm = F.normalize(query_emb, p=2, dim=1)
        g_norm = F.normalize(gallery_emb, p=2, dim=1)
        return 1 - torch.mm(q_norm, g_norm.t())
    else:
        raise ValueError("Unsupported metric")

def evaluate_rank1_map(dist_matrix, query_labels, gallery_labels):
    num_queries = dist_matrix.size(0)
    rank1 = 0
    ap_list = []
    query_labels = np.array(query_labels)
    gallery_labels = np.array(gallery_labels)
    for i in range(num_queries):
        distances = dist_matrix[i].cpu().numpy()
        sorted_indices = np.argsort(distances)
        matches = (gallery_labels[sorted_indices] == query_labels[i])
        if matches[0]:
            rank1 += 1
        num_relevant = matches.sum()
        if num_relevant == 0:
            continue
        precisions = []
        correct = 0
        for j, flag in enumerate(matches):
            if flag:
                correct += 1
                precisions.append(correct / (j + 1))
        ap_list.append(np.mean(precisions))
    rank1_accuracy = rank1 / num_queries
    mAP = np.mean(ap_list) if ap_list else 0
    return rank1_accuracy, mAP


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("pengcw1/market-1501")

print("Path to dataset files:", path)

In [ ]:


# --------------------------
# Define the Evaluation Dataset (if not already defined)
# --------------------------
class Market1501Dataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (str): Directory with images (e.g., query, bounding_box_test).
            transform: Preprocessing transforms.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        for file in os.listdir(root_dir):
            if file.endswith('.jpg'):
                id_str = file.split('_')[0]
                # Skip distractors/junk images (filenames starting with '-' or non-digit)
                if id_str.startswith('-') or not id_str.isdigit():
                    continue
                person_id = int(id_str)
                if person_id <= 0:
                    continue
                self.image_paths.append(os.path.join(root_dir, file))
                self.labels.append(person_id)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

# --------------------------
# Evaluation Functions (as provided earlier)
# --------------------------
def extract_embeddings(model, data_loader, device):
    model.eval()
    all_embeddings = []
    all_labels = []
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            embeddings = model(images)  # Expected shape: (B, 256)
            all_embeddings.append(embeddings.cpu())
            all_labels.extend(labels.numpy())
    all_embeddings = torch.cat(all_embeddings, dim=0)
    return all_embeddings, np.array(all_labels)

def compute_distance_matrix(query_emb, gallery_emb, metric='euclidean'):
    if metric == 'euclidean':
        return torch.cdist(query_emb, gallery_emb, p=2)
    elif metric == 'cosine':
        q_norm = F.normalize(query_emb, p=2, dim=1)
        g_norm = F.normalize(gallery_emb, p=2, dim=1)
        return 1 - torch.mm(q_norm, g_norm.t())
    else:
        raise ValueError("Unsupported metric")

def evaluate_rank1_map(dist_matrix, query_labels, gallery_labels):
    num_queries = dist_matrix.size(0)
    rank1 = 0
    ap_list = []
    query_labels = np.array(query_labels)
    gallery_labels = np.array(gallery_labels)
    for i in range(num_queries):
        distances = dist_matrix[i].cpu().numpy()
        sorted_indices = np.argsort(distances)
        matches = (gallery_labels[sorted_indices] == query_labels[i])
        if matches[0]:
            rank1 += 1
        num_relevant = matches.sum()
        if num_relevant == 0:
            continue
        precisions = []
        correct = 0
        for j, flag in enumerate(matches):
            if flag:
                correct += 1
                precisions.append(correct / (j + 1))
        ap_list.append(np.mean(precisions))
    rank1_accuracy = rank1 / num_queries
    mAP = np.mean(ap_list) if ap_list else 0
    return rank1_accuracy, mAP

# --------------------------
# Set Up Preprocessing Pipeline
# --------------------------
preprocess_pipeline = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet mean
        std=[0.229, 0.224, 0.225]    # ImageNet std
    )
])

# --------------------------
# Set Dataset Paths (Adjust based on your downloaded structure)
# --------------------------
dataset_path = "/kaggle/input/market-1501/Market-1501-v15.09.15"  # Change as needed
query_dir = os.path.join(dataset_path, "query")
gallery_dir = os.path.join(dataset_path, "bounding_box_test")

# --------------------------
# Create DataLoaders for Query and Gallery Sets
# --------------------------
query_dataset = Market1501Dataset(root_dir=query_dir, transform=preprocess_pipeline)
gallery_dataset = Market1501Dataset(root_dir=gallery_dir, transform=preprocess_pipeline)

query_loader = DataLoader(query_dataset, batch_size=32, shuffle=False, num_workers=2)
gallery_loader = DataLoader(gallery_dataset, batch_size=32, shuffle=False, num_workers=2)

# --------------------------
# Load the Saved Model
# --------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"

# Instantiate your model. For simplicity, this version uses only the ResNet + GNN branch.
# Make sure SimpleReIDModel is defined as in the previous code.
class SimpleReIDModel(nn.Module):
    def __init__(self, device='cuda'):
        super(SimpleReIDModel, self).__init__()
        # Assuming ResNetBackbone and GNNBranch are defined
        self.backbone = EfficientNetBackbone(pretrained=True)
        self.gnn_branch = GNNBranch(in_channels=1280, hidden_channels=512, out_channels=256, grid_size=(7,7))
        self.device = device

    def forward(self, x):
        x = x.to(self.device)
        feature_map = self.backbone(x)  # Shape: (B, 2048, 7, 7)
        embedding = self.gnn_branch(feature_map)  # Shape: (B, 256)
        return embedding

# Instantiate and load your saved model
model = SimpleReIDModel(device=device).to(device)
model_checkpoint = "/content/simple_reid_model.pth"  # Adjust path if needed
model.load_state_dict(torch.load(model_checkpoint, map_location=device))
model.eval()  # Set model to evaluation mode

# --------------------------
# Run Evaluation
# --------------------------
# 1. Extract embeddings for query and gallery sets
query_embeddings, query_labels = extract_embeddings(model, query_loader, device)
gallery_embeddings, gallery_labels = extract_embeddings(model, gallery_loader, device)

# 2. Compute the distance matrix (choose metric: 'euclidean' or 'cosine')
dist_matrix = compute_distance_matrix(query_embeddings, gallery_embeddings, metric='euclidean')

# 3. Evaluate Rank-1 and mAP
rank1_accuracy, mAP = evaluate_rank1_map(dist_matrix, query_labels, gallery_labels)
print("Rank-1 Accuracy: {:.2%}".format(rank1_accuracy))
print("mAP: {:.2%}".format(mAP))


Rank-1 Accuracy: 91.86%
mAP: 35.38%
